# <a href="http://www.redditscore.com">![redditscore](https://s3.us-east-2.amazonaws.com/redditscore/logo.png)</a> 
***A machine learning approach to predicting how badly you'll get roasted for your sub-par reddit comments.***

Alex Hartford & Trevor Hacker

### **Dataset**

Reddit comments from September, 2018 (<a href="http://files.pushshift.io/reddit/">source</a>).  This is well over 100gb of data.  We will likely only use a subset of the data but will ultimately try to use the entire dataset.

### **Objectives**

Create a linear regression model to predict the reddit score of a comment a user is considering posting. 

Stretch goal - narrow comment scoring down by subreddit, as comment popularity will differ between reddit communities.

Allow users to use this model with a publicly available <a href="http://www.redditscore.com/api">API</a> & <a href="http://www.redditscore.com">Website</a>.

Open source the project to allow further contributions if anyone is interested.

In [1]:
import numpy as np
import pandas as pd

print('Libraries loaded!')

Libraries loaded!


In [4]:
print('Loading memes...')
df = pd.read_json('https://s3.us-east-2.amazonaws.com/redditscore/25rows', lines=True)
print('Memes are fully operational!')

Loading memes...
Memes are fully operational!


In [5]:
print(df.dtypes)
df.head()

archived                            bool
author                            object
author_created_utc               float64
author_flair_background_color     object
author_flair_css_class            object
author_flair_richtext             object
author_flair_template_id          object
author_flair_text                 object
author_flair_text_color           object
author_flair_type                 object
author_fullname                   object
body                              object
can_gild                            bool
can_mod_post                        bool
collapsed                           bool
collapsed_reason                  object
controversiality                   int64
created_utc                        int64
distinguished                    float64
edited                              bool
gilded                             int64
gildings                          object
id                                object
is_submitter                        bool
link_id         

,archived,author,author_created_utc,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,permalink,removal_reason,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type
0,False,kmoz,1.302835e+09,,,[],None,CPI,dark,text,...,/r/LabVIEW/comments/9bs9tk/finding_local_maxim...,NaN,1538538086,1,True,False,LabVIEW,t5_2rk9b,r/LabVIEW,public
1,False,bignation24,1.535723e+09,,noob,"[{'e': 'text', 't': 'redditor for a day'}]",,redditor for a day,dark,richtext,...,/r/Bitcoin/comments/9by5gr/shorting_bitcoin_le...,NaN,1538538086,0,True,False,Bitcoin,t5_2s3qj,r/Bitcoin,public
2,False,Mercadi,1.462828e+09,None,None,[],None,None,None,text,...,/r/worldnews/comments/9bwe7w/lobbyist_says_he_...,NaN,1538538086,7,True,False,worldnews,t5_2qh13,r/worldnews,public
3,False,[deleted],NaN,,None,NaN,None,None,dark,NaN,...,/r/AteTheOnion/comments/9bxo16/how_despicable_...,NaN,1538538086,8,True,False,AteTheOnion,t5_3i1ww,r/AteTheOnion,public
4,False,surfer_ryan,1.378433e+09,None,None,[],None,None,None,text,...,/r/youseeingthisshit/comments/9bujgs/did_you_j...,NaN,1538538086,5,True,False,youseeingthisshit,t5_3dde0,r/youseeingthisshit,public


In [7]:
# Pulling out the fields that we need
rdf = df[['subreddit', 'body', 'score']]
rdf.head(25)

,subreddit,body,score
0,LabVIEW,"Seconding this, Ive used it on some pretty lar...",1
1,Bitcoin,I am not in debt and I shall rise again to inv...,0
2,worldnews,So that thin crowd was actually Russians? I wa...,7
3,AteTheOnion,[deleted],8
4,youseeingthisshit,I hope he ran away like a giggling school girl...,5
5,anime_irl,You can just tell that he's trying not to say ...,1
6,NintendoSwitch,Get Zelda,86
7,AskEconomics,[removed],2
8,mexico,[removed],1
9,nottheonion,Nobody likes a lingerer,2


Moving forward, since JSON format is taking roughly 54gb of the 109gb dataset just repeating headers, and we only actually need a handful of the columns, we're going to clean the data and convert it to csv to more efficiently store the data.  This will be a huge size and performance boost as we work on developing our model.